# advent of code 2024 - [day 1](https://adventofcode.com/2024/day/1)

## python solution

In [ ]:
def gen_lists(file='input.txt'):
    """Generates tuples of integers"""
    file = open(file, 'r')
    for _, line in enumerate(file):
        els = line.strip().split("   ")
        yield (int(el) for el in els)

list_1 = list(el for el, _ in gen_lists())
list_2 = list(el for _, el in gen_lists())
part1 = sum([abs(first-second) for first, second in zip(sorted(list_1), sorted(list_2))])
part2 = sum(x * list_2.count(x) for x in list_1)

part1, part2

## Neo4j-based solution

In [ ]:
import os

NEO4J_URI = os.environ['NEO4J_URI']
NEO4J_USERNAME = os.environ['NEO4J_USERNAME']
NEO4J_PASSWORD = os.environ['NEO4J_PASSWORD']


In [ ]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

from neo4j import GraphDatabase
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

import pandas as pd

### Parsing

In [ ]:
list_1 = list(el for el, _ in gen_lists())
list_2 = list(el for _, el in gen_lists())
list = [{'first': x, 'second': y} for x, y in zip(list_1, list_2)]

### Ingestion

In [ ]:
query_ingest = """
UNWIND $list AS row
WITH row.first AS first, row.second AS second
CREATE (:Location:List1 {loc: first}), (:Location:List2 {loc: second})
"""

gds.run_cypher(query_ingest, {"list":list})

### Sorting

In [ ]:
queries = [f"""
MATCH (x:List{i})
WITH x ORDER BY x.loc
WITH collect(x) AS xs
CALL apoc.nodes.link(xs, 'NEXT');
""" for i in [1, 2]]

for q in queries:
    gds.run_cypher(q, {})

### Iterative Matching

In [ ]:
def infer (rules, params={}):
    """
    This is a function you can use if you want to run a set of inference rules
    until a convergence is reached. why not use it in a RDF-like reasoning context?
    """
    counter = 0
    while True:
        counter += 1
        any_update = False
        for rule in rules:
            with driver.session(database="neo4j") as session:
                result = session.run(rule, params)
            any_update = any_update or result.consume().counters._contains_updates
        if not any_update:
            break

In [ ]:
matching_query = """
MATCH (x:List1&!Processed
WHERE NOT EXISTS {(:List1&!Processed)-[:NEXT]->(x)}),
(y:List2&!Processed
WHERE NOT EXISTS {(:List2&!Processed)-[:NEXT]->(y)})
MERGE (x)-[:MATCH]->(y)
SET x:Processed, y:Processed;
"""

rules =[matching_query]
infer(rules)

### The graph is built. Time for cypher queries.

In [ ]:
part1_query = """
MATCH (x)-[:MATCH]->(y)
WITH abs (x.loc - y.loc) AS dist
RETURN sum(dist) AS part1
"""

gds.run_cypher(part1_query, {})

In [ ]:
 # we'll need to MATCH list 2 elements on location value 
gds.run_cypher('CREATE INDEX list2_loc IF NOT EXISTS FOR (l:List2) ON (l.loc)')

In [ ]:
part2_query = """
MATCH (el1:List1)
WITH el1, count{
  MATCH (el2:List2 {loc: el1.loc})
  RETURN el2
} * el1.loc AS dist
RETURN sum (dist) AS part2
"""

gds.run_cypher(part2_query, {})